In [1]:
!pip install --upgrade langchain langchain-community streamlit python-dotenv requests PyPDF2 python-docx notebook ipykernel


In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access Groq API key
groq_api_key = os.getenv("GROQ_API_KEY")
print("Groq API Key loaded:", "Yes" if groq_api_key else "No")


Groq API Key loaded: Yes


In [4]:
import requests

def test_groq_api(api_key):
    url = "https://api.groq.com/openai/v1/models"
    headers = {"Authorization": f"Bearer {api_key}"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print("Groq API connection successful! Available models:")
        models = response.json().get("data", [])
        for model in models:
            print("-", model.get("id"))
    else:
        print(f"Failed to connect to Groq API: {response.status_code}")
        print(response.text)

# Run test
test_groq_api(groq_api_key)


Groq API connection successful! Available models:
- groq/compound
- meta-llama/llama-4-maverick-17b-128e-instruct
- qwen/qwen3-32b
- whisper-large-v3-turbo
- playai-tts-arabic
- gemma2-9b-it
- whisper-large-v3
- llama-3.1-8b-instant
- groq/compound-mini
- meta-llama/llama-prompt-guard-2-86m
- playai-tts
- meta-llama/llama-guard-4-12b
- llama-3.3-70b-versatile
- moonshotai/kimi-k2-instruct-0905
- allam-2-7b
- meta-llama/llama-prompt-guard-2-22m
- openai/gpt-oss-120b
- moonshotai/kimi-k2-instruct
- deepseek-r1-distill-llama-70b
- meta-llama/llama-4-scout-17b-16e-instruct
- openai/gpt-oss-20b


In [5]:
import langchain
import langchain_community

print("LangChain version:", langchain.__version__)
print("LangChain Community version:", langchain_community.__version__)


LangChain version: 0.3.27
LangChain Community version: 0.3.29


In [9]:
from langchain.llms.base import LLM
from pydantic import BaseModel, Field
import requests

class GroqTestLLM(LLM, BaseModel):
    api_key: str = Field(...)
    model_name: str = Field(default="gemma2-9b-it")
    temperature: float = Field(default=0.3)

    def _call(self, prompt: str, stop=None, **kwargs) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 50,
        }
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers=headers,
            json=payload,
        )
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"]

    @property
    def _llm_type(self) -> str:
        return "groq_test"

# Instantiate and test prompt
llm = GroqTestLLM(api_key=groq_api_key)
test_prompt = "What is the capital of India?"
print("LLM response:", llm._call(test_prompt))


LLM response: The capital of India is **New Delhi**. 

